In [1]:
import pandas as pd
import numpy as np
from DS_Planck_Unet import val_pix, east_val, west_val
from DS_models_stat import stat_orig_cats, stat_split_cats, do_all_stats, stat_cat, change_df
import seaborn as sns
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from DS_data_transformation import inter_cats

In [8]:
format_fun = lambda x: '{:.3f}'.format(x) if not (float(x).is_integer() or type(x) == type('')) else x
prm = 'N_cl - N_PSZ2'
files3 = { 
        'SZcat(AL)' : '/home/rt2122/Data/detected_cats/full_pz_all_found34_thr0.1_step8.csv',
        'SZcat(base)' : '/home/rt2122/Data/detected_cats/full_pz_rot28_thr0.1_step8.csv',
        'SZcat(y-map)' : '/home/rt2122/Data/SRGz/nilc_src_0.3deg_mar12_matched.csv',
        'PSZ2' : '/home/rt2122/Data/original_catalogs/csv/PSZ2.csv'
        }
names = ['SZcat(y-map)', 'SZcat(base)', 'SZcat(AL)', 'SZcat(gen)', 'PSZ2']

vals_default = {'SZcat(AL)' : 0.0, 'SZcat(base)' : 0.0, 'SZcat(y-map)' : -np.inf, 'SZcat(gen)' : '0.00', 'PSZ2' : ''}
vals_rel = {'SZcat(AL)' : 0.775, 'SZcat(base)' : 0.905, 'SZcat(y-map)' : 10.5, 'SZcat(gen)' : '0.81', 'PSZ2' : ''}
prm_lines = ['s/n >= {:.2f}', 'p_max >= {:.2f}', 'p_max >= {:.2f}', 'mpp_er >= {}', 'psz2.{}']

In [27]:
def calc_line(name, dict_cut, big_pix, prm_line, e_mode=False):
    recall = change_df(stat_cat({name : files3[name]}, big_pix=big_pix, dict_cut=dict_cut, 
                  match_dist=400/3600, spec_precision=['eROSITA', 'PSZ2_inter_all_true', 'PSZ2_inter_eROSITA', 'PSZ2', 'all_true']), 
                       format_s=format_fun)
    
    recall[prm] = recall['found_all_true'] - recall['found_PSZ2_inter_all_true']
    if e_mode:
        recall[prm] = recall['found_eROSITA'] - recall['found_PSZ2_inter_eROSITA']
        
    recall['prm'] = prm_line 
    return recall

In [28]:
#таблица 3 первая половина
vals = vals_default
files3['SZcat(gen)'] = '/home/rt2122/Data/gen_cats_er/er_gen_all_found_brcat_prec{}.csv'.format(vals['SZcat(gen)'])

df = []
big_pix = [41]
l_val = [-np.inf, np.inf]
b_val = [-np.inf, np.inf]
prm_list = ['PSZ2(z)', 'MCXC', 'ACT', 'all_true', 'precision', prm, 'all', 'prm']
prm_list1 = ['PSZ2(z)', 'MCXC', 'ACT', 'all_true', 'precision', prm, 'all', 'found_PSZ2']

dict_cut_list = [{'s/n' : [vals[names[0]], np.inf], 'l' : l_val, 'b' : b_val},
                 {'max_pred' : [vals[names[1]], np.inf], 'l' : l_val, 'b' : b_val},
                 {'max_pred' : [vals[names[2]], np.inf], 'l' : l_val, 'b' : b_val},
                 {'l' : l_val, 'b' : b_val},
                {'l' : l_val, 'b' : b_val}]

for name, dict_cut, prm_line in zip(names, dict_cut_list, prm_lines):
    df.append(calc_line(name=name, dict_cut=dict_cut, big_pix=big_pix, prm_line=prm_line.format(vals[name])))
df = pd.concat(df)

In [29]:
df[prm_list1]

,PSZ2(z),MCXC,ACT,all_true,precision,N_cl - N_PSZ2,all,found_PSZ2
SZcat(y-map),0.962,0.500,0.284,0.305,0.219,61,530,55
SZcat(base),0.943,0.500,0.268,0.295,0.302,57,364,53
SZcat(AL),0.943,0.515,0.293,0.321,0.279,67,430,53
SZcat(gen),0.943,0.529,0.345,0.363,0.181,84,764,54
PSZ2,1,0.382,0.113,0.144,1,0,58,58


In [30]:
print(df[prm_list1].to_latex())

\begin{tabular}{llllllrrr}
\toprule
{} & PSZ2(z) &   MCXC &    ACT & all\_true & precision &  N\_cl - N\_PSZ2 &  all &  found\_PSZ2 \\
\midrule
SZcat(y-map) &   0.962 &  0.500 &  0.284 &    0.305 &     0.219 &             61 &  530 &          55 \\
SZcat(base)  &   0.943 &  0.500 &  0.268 &    0.295 &     0.302 &             57 &  364 &          53 \\
SZcat(AL)    &   0.943 &  0.515 &  0.293 &    0.321 &     0.279 &             67 &  430 &          53 \\
SZcat(gen)   &   0.943 &  0.529 &  0.345 &    0.363 &     0.181 &             84 &  764 &          54 \\
PSZ2         &       1 &  0.382 &  0.113 &    0.144 &         1 &              0 &   58 &          58 \\
\bottomrule
\end{tabular}



In [31]:
#таблица 3 вторая половина
vals = vals_rel
files3['SZcat(gen)'] = '/home/rt2122/Data/gen_cats_er/er_gen_all_found_brcat_prec{}.csv'.format(vals['SZcat(gen)'])

df = []

dict_cut_list = [{'s/n' : [vals[names[0]], np.inf], 'l' : l_val, 'b' : b_val},
                 {'max_pred' : [vals[names[1]], np.inf], 'l' : l_val, 'b' : b_val},
                 {'max_pred' : [vals[names[2]], np.inf], 'l' : l_val, 'b' : b_val},
                 {'l' : l_val, 'b' : b_val},
                 {'l' : l_val, 'b' : b_val},
                ]

for name, dict_cut, prm_line in zip(names, dict_cut_list, prm_lines):
    df.append(calc_line(name=name, dict_cut=dict_cut, big_pix=big_pix, prm_line=prm_line.format(vals[name])))
df = pd.concat(df)

In [32]:
print(df[prm_list1].to_latex())

\begin{tabular}{llllllrrr}
\toprule
{} & PSZ2(z) &   MCXC &    ACT & all\_true & precision &  N\_cl - N\_PSZ2 &  all &  found\_PSZ2 \\
\midrule
SZcat(y-map) &   0.491 &  0.221 &  0.073 &    0.068 &         1 &              0 &   26 &          26 \\
SZcat(base)  &   0.585 &  0.324 &  0.070 &    0.091 &     0.829 &              3 &   41 &          31 \\
SZcat(AL)    &   0.717 &  0.353 &  0.088 &    0.115 &     0.878 &              5 &   49 &          38 \\
SZcat(gen)   &   0.717 &  0.353 &  0.088 &    0.115 &     0.878 &              5 &   49 &          38 \\
PSZ2         &       1 &  0.382 &  0.113 &    0.144 &         1 &              0 &   58 &          58 \\
\bottomrule
\end{tabular}



In [43]:
#таблица 4 первая половина
vals = vals_default
files3['SZcat(gen)'] = '/home/rt2122/Data/gen_cats_er/er_gen_all_found_brcat_prec{}.csv'.format(vals['SZcat(gen)'])

df = []
big_pix = list(range(48))
l_val = [0, 180]
b_val = [20, np.inf]
prm_list = ['PSZ2(z)', 'MCXC', 'ACT', 'eROSITA', 'precision_eROSITA', prm, 'all', 'prm']
prm_list1 = ['PSZ2(z)', 'MCXC', 'ACT', 'eROSITA', 'precision_eROSITA', prm, 'all', 'found_PSZ2']

dict_cut_list = [{'s/n' : [vals[names[0]], np.inf], 'l' : l_val, 'b' : b_val},
                 {'max_pred' : [vals[names[1]], np.inf], 'l' : l_val, 'b' : b_val},
                 {'max_pred' : [vals[names[2]], np.inf], 'l' : l_val, 'b' : b_val},
                 {'l' : l_val, 'b' : b_val},
                 {'l' : l_val, 'b' : b_val},
                ]

for name, dict_cut, prm_line in zip(names, dict_cut_list, prm_lines):
    df.append(calc_line(name=name, dict_cut=dict_cut, big_pix=big_pix, prm_line=prm_line.format(vals[name]), e_mode=True))
df = pd.concat(df)

In [44]:
print(df[prm_list1].to_latex())

\begin{tabular}{llllllrrr}
\toprule
{} & PSZ2(z) &   MCXC &    ACT & eROSITA & precision\_eROSITA &  N\_cl - N\_PSZ2 &    all &  found\_PSZ2 \\
\midrule
SZcat(y-map) &   0.965 &  0.427 &  0.220 &   0.101 &             0.266 &           1278 &   6862 &         645 \\
SZcat(base)  &   0.988 &  0.433 &  0.264 &   0.104 &             0.249 &           1352 &   7618 &         661 \\
SZcat(AL)    &   0.988 &  0.421 &  0.274 &   0.112 &             0.248 &           1500 &   8259 &         663 \\
SZcat(gen)   &   0.994 &  0.472 &  0.302 &   0.142 &             0.217 &           2059 &  12058 &         677 \\
PSZ2         &       1 &  0.301 &  0.102 &   0.033 &             0.812 &              0 &    692 &         692 \\
\bottomrule
\end{tabular}



In [45]:
#таблица 4 вторая половина
vals = vals_rel
files3['SZcat(gen)'] = '/home/rt2122/Data/gen_cats_er/er_gen_all_found_brcat_prec{}.csv'.format(vals['SZcat(gen)'])

df = []

dict_cut_list = [{'s/n' : [vals[names[0]], np.inf], 'l' : l_val, 'b' : b_val},
                 {'max_pred' : [vals[names[1]], np.inf], 'l' : l_val, 'b' : b_val},
                 {'max_pred' : [vals[names[2]], np.inf], 'l' : l_val, 'b' : b_val},
                 {'l' : l_val, 'b' : b_val},
                 {'l' : l_val, 'b' : b_val},
                ]

for name, dict_cut, prm_line in zip(names, dict_cut_list, prm_lines):
    df.append(calc_line(name=name, dict_cut=dict_cut, big_pix=big_pix, prm_line=prm_line.format(vals[name]), e_mode=True))
df = pd.concat(df)

In [46]:
print(df[prm_list1].to_latex())

\begin{tabular}{llllllrrr}
\toprule
{} & PSZ2(z) &   MCXC &    ACT & eROSITA & precision\_eROSITA &  N\_cl - N\_PSZ2 &  all &  found\_PSZ2 \\
\midrule
SZcat(y-map) &   0.391 &  0.160 &  0.044 &   0.014 &             0.801 &             23 &  282 &         209 \\
SZcat(base)  &   0.732 &  0.250 &  0.094 &   0.027 &             0.800 &             78 &  589 &         415 \\
SZcat(AL)    &   0.818 &  0.292 &  0.109 &   0.033 &             0.801 &            114 &  698 &         482 \\
SZcat(gen)   &   0.818 &  0.292 &  0.109 &   0.032 &             0.801 &            112 &  697 &         482 \\
PSZ2         &       1 &  0.301 &  0.102 &   0.033 &             0.812 &              0 &  692 &         692 \\
\bottomrule
\end{tabular}



In [37]:
df[prm_list1]

,PSZ2(z),MCXC,ACT,eROSITA,precision_eROSITA,N_cl - N_PSZ2,all,found_PSZ2
SZcat(y-map),0.391,0.160,0.044,0.014,0.801,22,282,209
SZcat(base),0.732,0.250,0.094,0.027,0.800,75,589,415
SZcat(AL),0.818,0.292,0.109,0.033,0.801,107,698,482
SZcat(gen),0.818,0.292,0.109,0.032,0.801,106,697,482
PSZ2,1,0.301,0.102,0.033,0.812,-1,692,692


In [40]:
PSZ2 = pd.read_csv('/home/rt2122/Data/original_catalogs/csv/PSZ2.csv')
eROSITA = pd.read_csv('/home/rt2122/Data/SRGz/clusters/clusters1_b20_edit.csv')
all_true = pd.read_csv('/home/rt2122/Data/original_catalogs/csv/other/PSZ2(z)_MCXC_ACT_united.csv')

In [41]:
PSZ2_inter_all_true = inter_cats(PSZ2.copy(), all_true.copy())
PSZ2_inter_eROSITA = inter_cats(eROSITA.copy(), PSZ2.copy())

In [42]:
PSZ2_inter_all_true.to_csv('/home/rt2122/Data/original_catalogs/csv/other/psz2_inter_all_true.csv', index=False)
PSZ2_inter_eROSITA.to_csv('/home/rt2122/Data/original_catalogs/csv/other/psz2_inter_eROSITA.csv', index=False)